In [ ]:
from music_pet import meta
from music_pet.services import vgmdb
from music_pet.audio import flac
import glob
import os
import sys
import subprocess

### Load Files

In [ ]:
WORKING_DIR = u'''/path/to/your/album'''

In [ ]:
INPUT_FILE = u'''CDImage.tta'''
CUE_FILE = u'''CDImage.cue'''
INI_FILE = u'''CDImage.ini'''

In [ ]:
OUTPUT_PATTERN = u"/path/to/your/music/<%(prefix)s >%(album)s< (%(suffix)s)>/<<%(discnumber)s->%(tracknumber)s >%(title)s.flac"

In [ ]:
os.chdir(WORKING_DIR)
!pwd

In [ ]:
album_list = meta.parse_cue(CUE_FILE)
album = album_list.values()[0]
print(album)

### Load album info from VGMDB

In [ ]:
album_info = vgmdb.get_album(10000000)  # get_album(id) id is the number on vgmdb detail page's url
print(album_info)

In [ ]:
vgmdb.update_album(album_info, album, lang=u"Japanese")
print(album)

### Load album info from Extra Info file

In [ ]:
extra_metas = meta.parse_ini(INI_FILE)

In [ ]:
for extra_meta in extra_metas:
    album.update_all_tracks(extra_meta)

### Before Convert

In [ ]:
print(str(album.detail()).decode("unicode_escape"))

### Convert

In [ ]:
flacs = flac.init_flacs(album, OUTPUT_PATTERN)

In [ ]:
PRE_COMMANDS = set()
CONV_COMMANDS = []
POST_COMMANDS = set()
TEMPWAV_MEMOIZE = {}

In [ ]:
for fl in flacs:
    fl.create_target_dir()
    fl.metadata.cover_picture = u"Front.jpg"
    if not fl.has_tag(u"@input_fullpath"):
        fl.set_input_file(fl.get_tag(u"original_file"))
    PRE_COMMANDS.add(fl.command_build_tempwav(memoize=TEMPWAV_MEMOIZE))
    POST_COMMANDS.add(fl.command_clear_tempwav())
    CONV_COMMANDS.append(fl.command())

In [ ]:
global_report = []
for cmd in PRE_COMMANDS:
    print(u"Executing:\n%s\n\n" % cmd)
    try:
        p = subprocess.check_output(cmd,
                                    shell=True,
                                    )
    except subprocess.CalledProcessError as ex:
        p = u"Process received an error! code=%s, output=%s" % (ex.returncode, ex.output)
        global_report.append((3, u"Process Error, code=%s" % ex.returncode, cmd))
    print(p)
    print(u"\n\n")
    
for cmd in CONV_COMMANDS:
    print(u"Executing:\n%s\n\n" % cmd)
    try:
        p = subprocess.check_output(cmd,
                                    shell=True,
                                    )
    except subprocess.CalledProcessError as ex:
        p = u"Process received an error! code=%s, output=%s" % (ex.returncode, ex.output)
        global_report.append((3, u"Process Error, code=%s" % ex.returncode, cmd))
    print(p)
    print(u"\n\n")
    
for cmd in POST_COMMANDS:
    print(u"Executing:\n%s\n\n" % cmd)
    try:
        p = subprocess.check_output(cmd,
                                    shell=True,
                                    )
    except subprocess.CalledProcessError as ex:
        p = u"Process received an error! code=%s, output=%s" % (ex.returncode, ex.output)
        global_report.append((3, u"Process Error, code=%s" % ex.returncode, cmd))
    print(p)
    print(u"\n\n")

In [ ]:
flacs[0].metadata.totaltracks